In [41]:
import os
import pandas as pd
import numpy as np

import re
from rapidfuzz import fuzz, process
from tqdm import tqdm

In [42]:
grants_path = r'C:\Projects\connecteddatahub\data\grants\cleaned_grants_2010_2018.csv'
grants_df = pd.read_csv(grants_path)
print(grants_df.shape)


(9262582, 10)


In [43]:
affiliation_path = r'C:\Projects\connecteddatahub\data\maps\cleaned_affiliation.csv'
affiliation_df = pd.read_csv(affiliation_path)
print(affiliation_df.head(4))
print(grants_df.columns)

   AffiliationId  carnegie_id  PrimarySample                  FullName  \
0       71965598     188429.0           True        Adelphi University   
1      181401687     131159.0           True       American University   
2      102298084     168740.0           True        Andrews University   
3           1000          NaN           True  Arizona Board of Regents   

   SystemId  
0       NaN  
1       NaN  
2       NaN  
3    1000.0  
Index(['year', 'recip_ein', 'recip_name', 'recip_city', 'recip_state',
       'recip_zip', 'amount', 'recip_status', 'text', 'type'],
      dtype='object')


In [44]:
diversity_path = r"C:\Projects\connecteddatahub\data\external\university_enrollment_race.csv"
diversity_df = pd.read_csv(diversity_path)
diversity_df = diversity_df.rename(columns={'year': 'Year'})
print(diversity_df.columns)
#merge the number of students into the df
affiliation_df = pd.merge(affiliation_df, diversity_df[['AffiliationId', 'student.size', 'Year']], how = "left", on="AffiliationId")
print(affiliation_df.head(5))
print(affiliation_df.columns)

Index(['carnegie_id', 'AffiliationId', 'Year', 'student.size',
       'student.enrollment.all', 'student.demographics.race_ethnicity.white',
       'student.demographics.race_ethnicity.black',
       'student.demographics.race_ethnicity.hispanic',
       'student.demographics.race_ethnicity.asian',
       'student.demographics.race_ethnicity.aian',
       'student.demographics.race_ethnicity.nhpi',
       'student.demographics.race_ethnicity.two_or_more',
       'student.demographics.race_ethnicity.non_resident_alien',
       'student.demographics.race_ethnicity.unknown',
       'student.demographics.race_ethnicity.white_non_hispanic',
       'student.demographics.race_ethnicity.black_non_hispanic',
       'student.demographics.race_ethnicity.asian_pacific_islander',
       'student.demographics.race_ethnicity.aian_prior_2009',
       'student.demographics.race_ethnicity.hispanic_prior_2009',
       'student.demographics.race_ethnicity.unknown_2000',
       'student.demographics.race_e

In [ ]:
years = ["1999", "2000", "2002", "2005", "2007", "2008", "2009", "2010", "2011", "2013", "2018"]

year_df = pd.DataFrame({'year': years})
affiliation_years = affiliation_df.merge(year_df, how = 'cross')
print(affiliation_years.columns)

   AffiliationId  carnegie_id  PrimarySample            FullName  SystemId  \
0       71965598     188429.0           True  Adelphi University       NaN   
1       71965598     188429.0           True  Adelphi University       NaN   
2       71965598     188429.0           True  Adelphi University       NaN   
3       71965598     188429.0           True  Adelphi University       NaN   
4       71965598     188429.0           True  Adelphi University       NaN   
5       71965598     188429.0           True  Adelphi University       NaN   
6       71965598     188429.0           True  Adelphi University       NaN   
7       71965598     188429.0           True  Adelphi University       NaN   
8       71965598     188429.0           True  Adelphi University       NaN   
9       71965598     188429.0           True  Adelphi University       NaN   

   student.size_x    Year  student.size_y  
0          3378.0  2001.0             NaN  
1          3726.0  2002.0             NaN  
2        

In [46]:
tqdm.pandas()  # enable tqdm for pandas apply

threshold = 85

def normalize_name(s):
    """Lowercase, strip, and remove punctuation/suffixes."""
    if pd.isna(s):
        return ""
    s = s.lower().strip()
    s = re.sub(r"[^\w\s]", "", s)
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r"\b(jr|sr|iii|ii)\b", "", s)
    return s.strip()


grants_df["year"] = grants_df["year"].astype(str)
affiliation_years["year"] = affiliation_years["year"].astype(str)

grants_df["recip_name_norm"] = grants_df["recip_name"].apply(normalize_name)
affiliation_years["fullname_norm"] = affiliation_years["FullName"].apply(normalize_name)



def match_name_to_grants(row, grants_subset):
    """Find the best matching recipient name for a given FullName within the same year."""
    name = row["fullname_norm"]
    if not name or grants_subset.empty:
        return None
    # direct substring check first
    substring_hits = grants_subset[
        grants_subset["recip_name_norm"].str.contains(name, na=False)
    ]
    if not substring_hits.empty:
        # if substring match exists, just return the first one (names are clean)
        return substring_hits.iloc[0]["recip_name_norm"]

    # otherwise, use fuzzy match
    match = process.extractOne(
        name,
        grants_subset["recip_name_norm"],
        scorer=fuzz.token_sort_ratio,
    )
    if match and match[1] >= threshold:
        return match[0]
    return None


# Perform matching year-by-year for speed
matched_chunks = []
years_iter = list(affiliation_years["year"].unique())

for yr in tqdm(years_iter, desc="Matching by year"):
    grants_subset = grants_df[grants_df["year"] == yr]
    sub_aff = affiliation_years[affiliation_years["year"] == yr].copy()
    sub_aff["matched_name"] = sub_aff.progress_apply(
        lambda r: match_name_to_grants(r, grants_subset), axis=1
    )
    matched_chunks.append(sub_aff)

affiliation_years = pd.concat(matched_chunks, ignore_index=True)


# Merge the matched results with grant amounts
merged = affiliation_years.merge(
    grants_df[["recip_name_norm", "year", "amount"]],
    how="left",
    left_on=["matched_name", "year"],
    right_on=["recip_name_norm", "year"]
)

# Aggregate number of grants and total funding
grant_stats = (
    merged.groupby(["FullName", "year"], as_index=False)
    .agg(
        num_grants=("amount", "count"),
        total_funding=("amount", "sum")
    )
)

# Reattach aggregated results to affiliation_years (keeping all original columns)
affiliation_years = affiliation_years.merge(
    grant_stats,
    on=["FullName", "year"],
    how="left"
)

affiliation_years["num_grants"] = affiliation_years["num_grants"].fillna(0).astype(int)
affiliation_years["total_funding"] = affiliation_years["total_funding"].fillna(0)

print(affiliation_years[[
    "AffiliationId", "SystemId", "carnegie_id", "PrimarySample",
    "FullName", "year", "num_grants", "total_funding"
]].head(20))


KeyError: 'year'

In [ ]:
grant_stats_filtered = affiliation_years[affiliation_years["year"].astype(int) >= 2010]


grant_stats_filtered.to_csv(r'C:\Projects\connecteddatahub\data\grants\university_grants.csv', index = False)